In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
sys.path.insert(0,"/ocean/projects/asc170022p/nmurali/projects/CounterfactualExplainer/MIMICCX-Chest-Explainer/Classifier/torchxrayvision_")
import inspect
import random
import pickle
from tqdm import tqdm
from skimage.io import imread, imsave
sys.path.insert(0,"/ocean/projects/asc170022p/nmurali/projects/CounterfactualExplainer/MIMICCX-Chest-Explainer/Classifier/torchxrayvision_")
import torchxrayvision as xrv
import torchvision
from torch.utils import data
import torch
import yaml
import sklearn
sys.path.insert(0,'/jet/home/nmurali/asc170022p/nmurali/projects/augmentation_by_explanation_eccv22/Classifier')
import datasets
from torch.utils.data import Dataset
sys.path.insert(0,"/ocean/projects/asc170022p/nmurali/projects/CounterfactualExplainer/MIMICCX-Chest-Explainer/stylegan2Pytorch")
from swagan_updatedEGC import Generator, Discriminator

In [ ]:
# user hyperparams
config_path = '/jet/home/nmurali/asc170022p/nmurali/projects/augmentation_by_explanation_eccv22/Configs/Classifier/DenseNet_Skin_ln0p50.yaml'
gan_config_path = '/jet/home/nmurali/asc170022p/nmurali/projects/augmentation_by_explanation_eccv22/Configs/Explainer/styleGAN_Skin_ln0p50.yaml'
main_dir = '/jet/home/nmurali/asc170022p/nmurali/projects/augmentation_by_explanation_eccv22/'
cls_final_path = os.path.join(main_dir,'Output/HAM/Classifier_Seed_0_Dropout_0.0_LS_False_MU_False_FL_False_skin_ln0p50/HAM-densenet169-HAM_256-best-auc0.5388.pt')
gan_dir = os.path.join(main_dir, 'Output/StyleGAN/HAM_ln0p50/checkpoint/083000.pt')
save_dir = os.path.join(main_dir, 'Output/ACE/da_skin_ln0p50/')
data_dir_DA = os.path.join(save_dir, 'augmented_data')

In [ ]:
# support functions

class center_crop(object):
    def crop_center(self, img):
        _, y, x = img.shape
        crop_size = np.min([y,x])
        startx = x // 2 - (crop_size // 2)
        starty = y // 2 - (crop_size // 2)
        return img[:, starty:starty + crop_size, startx:startx + crop_size]
    
    def __call__(self, img):
        return self.crop_center(img)

class normalize(object):
    def normalize_(self, img, maxval=255):
        img = (img)/(maxval)
        return img
    
    def __call__(self, img):
        return self.normalize_(img)

def print_output(pred):
    pred = np.round(pred,2)
    output = ''
    for i in range(0, 2):#pred.shape[0]):
        #if pred[i] > 0.5:
        output += pathologies[i] + ': ' + str(pred[i]) + '\n'
    return output

def requires_grad(model, flag=True):
    for p in model.parameters():
        p.requires_grad = flag
        
def uniform_binning(y_conf,bin_size=0.10):
    upper_bounds = np.arange(bin_size, 1+bin_size, bin_size)
    lower_bounds = upper_bounds- bin_size
    y_bin = []
    n_bins = len(upper_bounds)
    for s in y_conf:
        if (s <= upper_bounds[0]) :
            y_bin.append(0)
        elif (s > lower_bounds[n_bins-1]) :
            y_bin.append(n_bins-1)
        else:
            for i in range(1,n_bins-1):
                if (s > lower_bounds[i]) & (s <=upper_bounds[i] ):
                    y_bin.append(i)
                    break
    y_bin = np.asarray(y_bin)
    return y_bin

def compute_ece(y_pred,y_true ):
    #y_true = np.append(y_true, 1 - y_true,axis=1)
    #y_pred = np.append(y_pred, 1 - y_pred,axis=1)
    #y_true = np.argmax(y_true, axis=1)
    y_predicted_label = np.argmax(y_pred,axis=1)
    y_conf = np.max(y_pred,axis=1)
    
    y_bin = uniform_binning(y_conf,bin_size=0.10)
    n_bins = np.max(y_bin)+1
    N = y_bin.shape[0]
    ece = 0
    for b in range(n_bins):
        index = np.where(y_bin == b)
        y_pred_b = y_predicted_label[index]
        y_true_b = y_true[index]
        y_conf_b = y_conf[index]
        if y_pred_b.shape[0] == 0:
            ece +=0 
        else:
            acc = np.sum(y_pred_b == y_true_b)/y_pred_b.shape[0]
            c = y_pred_b.shape[0]
            conf = np.mean(y_conf_b)
            ece +=  np.abs(acc - conf) * (c)
    return ece/N


class SKIN(Dataset):
    def __init__(self, file_names, transforms, seed=0):

        super(SKIN, self).__init__()
        np.random.seed(seed)  # Reset the seed so all runs are the same.               
        self.transform = transforms             

        self.file_names = file_names
    
    def __len__(self):
        return self.file_names.shape[0]

    def __getitem__(self, idx):          
        img_path = self.file_names[idx]
        img = imread(img_path)
        img = self.transform(img)
        return {"img":img,  "idx":idx, "file_name":img_path}

In [ ]:
config = yaml.safe_load(open(config_path)) 
config['class_names'] = config['class_names'].split(',')

# ============= Dataset ====================
df = pd.read_csv(config['data_file'])
try:
    df_train = df.loc[(df[config['column_name_split']]==1)]
    train_inds = np.asarray(df_train.index)
    df_train = df.loc[(df[config['column_name_split']]==0)]
    test_inds = np.asarray(df_train.index)
    print("train: ", train_inds.shape, "test: ", test_inds.shape)
except:
    print("The data_file doesn't have a train column, hence we will randomly split the entire dataset to have 15% samples as validation set.")
    train_inds=np.empty([])
    test_inds=np.empty([])

if config['dataset'] == 'AFHQ':
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),\
        torchvision.transforms.Resize((config['size'], config['size'])), 
        torchvision.transforms.RandomHorizontalFlip(p=config['data_aug_hf']),
        torchvision.transforms.ToTensor()
    ])
    
    dataset = datasets.AFHQ_Dataset(csvpath=config['data_file'], class_names=config['class_names'], transform=transforms, seed=config['seed'])
elif config['dataset'] == 'HAM':
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),\
        torchvision.transforms.Resize((config['size'], config['size'])), 
        torchvision.transforms.RandomHorizontalFlip(p=config['data_aug_hf']),
        torchvision.transforms.RandomVerticalFlip(p=config['data_aug_hf']),
        torchvision.transforms.ToTensor()
    ])
    dataset = datasets.HAM_Dataset(imgpath=config['imgpath'],csvpath=config['data_file'],class_names=config['class_names'],unique_patients=False, transform=transforms, seed=config['seed'])

elif config['dataset'] == 'Dirty_MNIST':
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),\
        torchvision.transforms.Resize((config['size'], config['size'])), 
        torchvision.transforms.ToTensor()
    ])
    train_inds = datasets.DIRTY_MNIST_Dataset(csvpath=config['data_file'], transform=transforms, class_names=config['class_names'], seed=config['seed'])
    test_inds = datasets.DIRTY_MNIST_Dataset(csvpath=config['data_file_test'], transform=transforms, class_names=config['class_names'], seed=config['seed'])
    dataset = None

elif config['dataset'] == 'CelebA':
    transforms = torchvision.transforms.Compose([
        torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((config['size'], config['size'])),
        torchvision.transforms.CenterCrop(config['center_crop']),
        torchvision.transforms.Resize((config['size'], config['size'])),
        torchvision.transforms.RandomHorizontalFlip(p=config['data_aug_hf']),
        torchvision.transforms.ToTensor()
    ])
    dataset = datasets.CelebA(imgpath=config['imgpath'],  csvpath=config['data_file'], class_names=config['class_names'], transform=transforms, seed=config['seed'])

elif config['dataset'] == 'Stanford-CHEX':
    transforms = torchvision.transforms.Compose([
        #torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((config['size'], config['size'])),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Lambda(center_crop()),
        torchvision.transforms.Lambda(normalize())
    ])
    train_inds = datasets.CheX_Dataset(imgpath=config['imgpath'], csvpath=config['data_file'], class_names=config['class_names'], transform=transforms, seed=config['seed'])
    test_inds = datasets.CheX_Dataset(imgpath=config['imgpath'], csvpath=config['data_file_test'], class_names=config['class_names'], transform=transforms, seed=config['seed'])
    dataset = None

elif config['dataset'] == 'MIMIC-CXR':
    transforms = torchvision.transforms.Compose([
        #torchvision.transforms.ToPILImage(),
        torchvision.transforms.Resize((config['size'], config['size'])),
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Lambda(center_crop()),
        torchvision.transforms.Lambda(normalize())
    ])
    dataset = datasets.MIMIC_Dataset(imgpath=config['imgpath'], csvpath=config['data_file'], class_names=config['class_names'], transform=transforms, seed=config['seed'])

In [ ]:
if dataset is not None:
    if len(train_inds.shape) == 0:
        gss = sklearn.model_selection.GroupShuffleSplit(train_size=0.85,test_size=0.15, random_state=config['seed'])
        train_inds, test_inds = next(gss.split(X=range(len(dataset)), groups=dataset.csv.patientid))
        train_dataset = datasets.SubsetDataset(dataset, train_inds)
        valid_dataset = datasets.SubsetDataset(dataset, test_inds)
        np.save(os.path.join(config['output_dir'], 'train.npy'), train_inds)
        np.save(os.path.join(config['output_dir'], 'validation.npy'), test_inds)
    else:
        train_dataset = datasets.SubsetDataset(dataset, train_inds)
        valid_dataset = datasets.SubsetDataset(dataset, test_inds)
    print("Train: ", train_dataset)
    print("Validation: ", valid_dataset)
else:
    train_dataset = train_inds
    valid_dataset = test_inds


# Dataloader
train_loader = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=config['batch_size'],
                                            shuffle=config['shuffle'],
                                            num_workers=config['threads'], 
                                            pin_memory=config['cuda'])
test_loader = torch.utils.data.DataLoader(valid_dataset,
                                            batch_size=config['batch_size'],
                                            shuffle=config['shuffle'],
                                            num_workers=config['threads'], 
                                            pin_memory=config['cuda'])

In [ ]:
num_cls = config['num_classes']
batch_size = 8
device = "cuda"
pathologies = config['class_names']
img_size = config['size']
if not os.path.exists(data_dir_DA):
    os.makedirs(data_dir_DA)
data_dir_save = os.path.join(save_dir, 'save')
if not os.path.exists(data_dir_save):
    os.makedirs(data_dir_save)

In [ ]:
for batch in train_loader:
    break
print(batch['img'].shape, batch['lab'].shape)
temp = np.asarray(batch['img'].detach().cpu())
label = np.asarray(batch['lab'].detach().cpu())
temp = np.moveaxis(temp, 1,3)
fig = plt.figure(figsize= (15,3))
for i in range(batch_size):
    plt.subplot(1,10,i+1)
    plt.imshow(temp[i])
    plt.title(label[i])
    plt.axis('off')
plt.subplots_adjust(wspace=0, hspace=0)
plt.title('Train Data')
plt.show()

# Baseline classifier performance on train dataset

In [ ]:
seed = config['seed']
print(seed)
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
classifier = xrv.models.DenseNet(num_classes=config['num_classes'], in_channels=config['channel'], drop_rate = config['drop_rate'], \
                                 weights = cls_final_path, return_logit=True,\
                                 **xrv.models.get_densenet_params(config['model'])).to(device) 

In [ ]:
all_real_pred = np.empty([0])
all_names = np.empty([0])
all_true_labels = np.empty([0]) #N
requires_grad(classifier, False)
for b_idx, batch in enumerate(tqdm(train_loader)):
    targets = np.asarray(batch['lab'].detach().cpu())
    names = np.asarray(batch['file_name'])
    real_img = batch['img']
    real_img = real_img.to(device)
    real_pred_cls,_ = classifier(real_img)
    real_pred_cls = torch.sigmoid(real_pred_cls) 
    if all_real_pred.shape[0] == 0:
        all_real_pred = np.asarray(real_pred_cls.detach().cpu())
        all_names = names
        all_true_labels = targets
    else:
        all_real_pred = np.append(all_real_pred, np.asarray(real_pred_cls.detach().cpu()), axis=0)
        all_names = np.append(all_names, names, axis=0)
        all_true_labels = np.append(all_true_labels, targets, axis=0)
    if b_idx>500:
        break
print(all_real_pred.shape, all_names.shape, all_true_labels.shape)

In [ ]:
all_true_class = np.argmax(all_true_labels,axis=1)

In [ ]:
conf = np.max(all_real_pred, axis=1)
pe = np.sum(-1*(all_real_pred * np.log2(all_real_pred)),1)
pe = pe/np.max(pe) #normalize

In [ ]:
fig = plt.figure(figsize= (10,3))
plt.subplot(1,2,1)
plt.hist(conf, label = 'UiD (10K)',  bins=40)
plt.title('Predicted confidence on AFHQ train-set')
plt.legend()
plt.subplot(1,2,2)
plt.hist(pe, label = 'UiD (10K)', bins=40)
plt.title('Predicted entropy')
plt.legend()
plt.show()

# Select a subset of real-train-set from where we will derive DA

In [ ]:
# consider only those images in class NV, Mel, bkl
all_real_pred.shape, all_names.shape, all_true_labels.shape

In [ ]:
mean_confidence = np.max(all_real_pred, axis=1)
index = np.where(mean_confidence<0.7)
index[0].shape

In [ ]:
np.random.shuffle(index[0])
index = (index[0][:2000],)
index[0].shape

In [ ]:
create_DA_real_names = all_names[index]
create_DA_real_pred = all_real_pred[index]
create_DA_real_labels = all_true_labels[index]
print(create_DA_real_names.shape, create_DA_real_labels.shape, create_DA_real_pred.shape)

In [ ]:
np.save(os.path.join(data_dir_DA, 'create_DA_real_names.npy'),create_DA_real_names)
np.save(os.path.join(data_dir_DA, 'create_DA_real_pred.npy'),create_DA_real_pred)
np.save(os.path.join(data_dir_DA, 'create_DA_real_labels.npy'),create_DA_real_labels)


In [ ]:
create_DA_real_names = np.load(os.path.join(data_dir_DA, 'create_DA_real_names.npy'))
create_DA_real_pred = np.load(os.path.join(data_dir_DA, 'create_DA_real_pred.npy'))
create_DA_real_labels = np.load(os.path.join(data_dir_DA, 'create_DA_real_labels.npy'))
print(create_DA_real_names.shape, create_DA_real_labels.shape, create_DA_real_pred.shape)

# Select a random set of images to make real-pool

In [ ]:
index = np.where(mean_confidence>0.7)

In [ ]:
consider_names = all_names[index]
consider_images_pred = all_real_pred[index]
consider_images_label = all_true_labels[index]
print(consider_names.shape, consider_images_pred.shape, consider_images_label.shape)

In [ ]:
random_index = np.random.choice(range(consider_names.shape[0]),2000)
random_index.shape,all_names.shape[0]

In [ ]:
consider_names

In [ ]:
consider_names = consider_names[random_index]
consider_images_pred = consider_images_pred[random_index]
consider_images_label = consider_images_label[random_index]
print(consider_names.shape, consider_images_pred.shape, consider_images_label.shape)

In [ ]:
plt.hist(consider_images_pred[:,1],bins=40)
plt.show()

In [ ]:
np.save(os.path.join(data_dir_DA, 'consider_names.npy'),consider_names)
np.save(os.path.join(data_dir_DA, 'consider_images_pred.npy'),consider_images_pred)
np.save(os.path.join(data_dir_DA, 'consider_images_label.npy'),consider_images_label)

In [ ]:
consider_names,data_dir_DA

# Derive DA

In [ ]:
create_DA_real_names = np.load(os.path.join(data_dir_DA, 'create_DA_real_names.npy'))
create_DA_real_pred = np.load(os.path.join(data_dir_DA, 'create_DA_real_pred.npy'))
create_DA_real_labels = np.load(os.path.join(data_dir_DA, 'create_DA_real_labels.npy'))
print(create_DA_real_names.shape, create_DA_real_labels.shape, create_DA_real_pred.shape)

In [ ]:
y_true = np.argmax(create_DA_real_labels,axis=1)
np.unique(y_true,return_counts=True)

In [ ]:
# Create fake images and preds

gan_config = yaml.safe_load(open(gan_config_path)) 

create_DA_real_names = np.load(os.path.join(data_dir_DA, 'create_DA_real_names.npy'))
suffix = '_DA'
print(create_DA_real_names.shape)

temp_batch_size = 2
cond = np.asarray([0.2, 0.4, 0.6, 0.8 ])
n_bins = cond.shape[0]
cond = [cond for i in range(temp_batch_size)]
cond = np.stack(cond, 0)
cond = np.ravel(cond)
cond, cond.shape

skin_dataset = SKIN(file_names=create_DA_real_names,transforms=transforms)
skin_loader = data.DataLoader(
    skin_dataset,
    batch_size=2,
    drop_last=True,
    shuffle=False
)

generator = Generator(config['size'], 512, config['num_classes'], 8, channel_multiplier=2).to("cuda")
discriminator = Discriminator(config['size'], channel_multiplier=2, concate_size=gan_config['concate_size']).to("cuda")

print("load model:", gan_dir)
ckpt = torch.load(gan_dir, map_location=lambda storage, loc: storage)
generator.load_state_dict(ckpt["g"])
discriminator.load_state_dict(ckpt["d"])
if torch.cuda.is_available():
    generator.cuda()
    discriminator.cuda()

all_fake_imgs = np.empty([0])
all_fake_pred = np.empty([0])
all_fake_pred_d_s = np.empty([0])
all_fake_pred_d = np.empty([0])
all_fake_cond = np.empty([0])
all_real_pred = np.empty([0])
all_real_imgs = np.empty([0])
all_real_imgs_d = np.empty([0])
all_real_imgs_d_s = np.empty([0])
for samples in tqdm(skin_loader): 
    real_imgs = samples['img']
    real_imgs = real_imgs.to(device)
    cls_preds, cls_pred_f = classifier(real_imgs)
    cls_preds = torch.sigmoid(cls_preds)
    cls_pred_d = discriminator(real_imgs, cls_pred_f)
    cls_pred_d_s = torch.sigmoid(cls_pred_d)

    real_img_np = np.asarray(real_imgs.detach().cpu())
    cls_pred_np = np.asarray(cls_preds.detach().cpu())
    cls_pred_d_np = np.asarray(cls_pred_d.detach().cpu())
    cls_pred_d_s_np = np.asarray(cls_pred_d_s.detach().cpu())

    current_image = np.repeat(real_img_np, n_bins,axis=0)
    current_image = torch.Tensor(current_image).to(device)
    current_pred = np.repeat(cls_pred_np, n_bins,axis=0)


    cond = np.random.random(4) #np.asarray([0.2, 0.4, 0.6, 0.8 ])
    n_bins = cond.shape[0]
    cond = [cond for i in range(temp_batch_size)]
    cond = np.stack(cond, 0)
    cond = np.ravel(cond)

    current_pred[:,0] = cond
    current_pred[:,1] = 1-cond
    current_pred = torch.Tensor(current_pred).to(device)

    recon_img, _ = generator(current_image, current_pred, return_latents=False)
    fake_pred, fake_pred_f = classifier(recon_img)
    fake_pred = torch.sigmoid(fake_pred)
    fake_pred_d = discriminator(recon_img, fake_pred_f)
    fake_pred_d_s = torch.sigmoid(fake_pred_d)

    recon_img = np.asarray(recon_img.detach().cpu())
    fake_pred = np.asarray(fake_pred.detach().cpu())
    current_pred = np.asarray(current_pred.detach().cpu())
    fake_pred_d = np.round(np.asarray(fake_pred_d.detach().cpu()),3)
    fake_pred_d_s = np.round(np.asarray(fake_pred_d_s.detach().cpu()),3)
    if all_real_imgs.shape[0] == 0:
        all_fake_imgs = recon_img # generated image with given cond
        all_fake_pred = fake_pred #classification outcome of the generated image
        all_fake_pred_d = fake_pred_d #discriminator outcome of the generated image
        all_fake_pred_d_s = fake_pred_d_s
        all_fake_cond = current_pred #cond used for generation
        all_real_pred = cls_pred_np #real image pred
        all_real_imgs = real_img_np
        all_real_imgs_d = cls_pred_d_np # real image discriminator outcome
        all_real_imgs_d_s = cls_pred_d_s_np
    else:
        all_fake_imgs =np.append(all_fake_imgs, recon_img,axis=0)
        all_fake_pred = np.append(all_fake_pred, fake_pred,axis=0)
        all_fake_pred_d = np.append(all_fake_pred_d, fake_pred_d,axis=0)
        all_fake_pred_d_s = np.append(all_fake_pred_d_s, fake_pred_d_s,axis=0)
        all_fake_cond = np.append(all_fake_cond, current_pred,axis=0)
        all_real_pred = np.append(all_real_pred, cls_pred_np,axis=0)
        all_real_imgs = np.append(all_real_imgs, real_img_np,axis=0)
        all_real_imgs_d = np.append(all_real_imgs_d, cls_pred_d_np,axis=0)
        all_real_imgs_d_s = np.append(all_real_imgs_d_s, cls_pred_d_s_np,axis=0)



np.save(os.path.join(data_dir_save, 'all_fake_imgs'+suffix+'.npy'), all_fake_imgs)
np.save(os.path.join(data_dir_save, 'all_fake_pred'+suffix+'.npy'), all_fake_pred)
np.save(os.path.join(data_dir_save, 'all_fake_pred_d'+suffix+'.npy'), all_fake_pred_d)
np.save(os.path.join(data_dir_save, 'all_fake_pred_d_s'+suffix+'.npy'), all_fake_pred_d_s)
np.save(os.path.join(data_dir_save, 'all_fake_cond'+suffix+'.npy'), all_fake_cond)
np.save(os.path.join(data_dir_save, 'all_real_pred'+suffix+'.npy'), all_real_pred)
np.save(os.path.join(data_dir_save, 'all_real_imgs'+suffix+'.npy'), all_real_imgs)
np.save(os.path.join(data_dir_save, 'all_real_imgs_d'+suffix+'.npy'), all_real_imgs_d)
np.save(os.path.join(data_dir_save, 'all_real_imgs_d_s'+suffix+'.npy'), all_real_imgs_d_s)

In [ ]:
#Read DA
data_dir_save = os.path.join(save_dir, 'save')
all_fake_imgs = np.load(os.path.join(data_dir_save, 'all_fake_imgs_DA.npy'))
all_fake_pred = np.load(os.path.join(data_dir_save, 'all_fake_pred_DA.npy'))
all_fake_pred_d = np.load(os.path.join(data_dir_save, 'all_fake_pred_d_DA.npy'))
all_fake_pred_d_s = np.load(os.path.join(data_dir_save, 'all_fake_pred_d_s_DA.npy'))
all_fake_cond = np.load(os.path.join(data_dir_save, 'all_fake_cond_DA.npy'))
all_real_pred = np.load(os.path.join(data_dir_save, 'all_real_pred_DA.npy'))
all_real_imgs = np.load(os.path.join(data_dir_save, 'all_real_imgs_DA.npy'))
all_real_imgs_d = np.load(os.path.join(data_dir_save, 'all_real_imgs_d_DA.npy'))
all_real_imgs_d_s = np.load(os.path.join(data_dir_save, 'all_real_imgs_d_s_DA.npy'))
print(all_fake_imgs.shape, all_fake_pred.shape, all_fake_pred_d.shape, all_fake_cond.shape)
print(all_real_pred.shape, all_real_imgs.shape, all_real_imgs_d.shape)

In [ ]:
plt.hist(all_real_imgs_d_s[:,0], label = 'real', bins = 40,alpha=0.5)
plt.hist(all_fake_pred_d_s[:,0], label = 'fake', bins = 40,alpha=0.5)
plt.legend()
plt.xlim([0,1])
plt.show()

In [ ]:
all_real_imgs_view = np.moveaxis(all_real_imgs, 1,3)
all_fake_imgs_view = np.moveaxis(all_fake_imgs, 1,3)
print(all_real_imgs_view.shape, all_fake_imgs_view.shape)

In [ ]:
n_bins = int(all_fake_imgs.shape[0]/all_real_pred.shape[0])
n_bins

In [ ]:
for i in range(0, 10):
    fig = plt.figure(figsize=(20,4))
    plt.subplot(1,n_bins+1,1)
    plt.imshow(all_real_imgs_view[i,:,:,:])
    plt.title(print_output(all_real_pred[i]))
    plt.ylabel(all_real_imgs_d_s[i][0])
    for j in range(n_bins):
        plt.subplot(1,n_bins+1,j+2)
        plt.imshow(all_fake_imgs_view[i*n_bins + j])
        plt.ylabel(all_fake_pred_d_s[i*n_bins + j][0])
        plt.title(print_output(all_fake_cond[i*n_bins + j]))
        plt.xlabel(print_output(all_fake_pred[i*n_bins + j]))
    
    
    plt.show()

In [ ]:
plt.hist(all_fake_pred[:,0],bins=40, label = 'Predicted probability on Fake images', alpha=0.3)
#plt.hist(all_real_pred[:,0],bins=40, label = 'Predicted probability on Real images used to create the fake images',alpha=0.3)
plt.hist(all_fake_cond[:,0],bins=40, label = 'Condition used for creating Fake images',alpha=0.3)
plt.title("Fake images created for Data Augmentation")
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

In [ ]:
#save fake-pool as separate files
data_dir_save_fake = os.path.join(save_dir, 'save', 'fake')
try:
    os.makedirs(data_dir_save_fake)
except:
    pass
all_rows = []
for i in range(all_fake_imgs.shape[0]):
    file_name = os.path.join(data_dir_save_fake, 'fake_' + str(i) + '.npy')
    np.save(file_name, all_fake_imgs[i])
    row = {}
    row['names'] = file_name
    for j in range(len(pathologies)):
        row[pathologies[j]] = all_fake_cond[i][j]
    all_rows.append(row)
df_fake = pd.DataFrame(data=all_rows)
print(df_fake.shape)
df_fake.head(2)

In [ ]:
df_fake.to_csv(os.path.join(data_dir_save, 'fake.csv'),index=None)

In [ ]:
data_dir_save

# Save some real-images

In [ ]:
consider_names = np.load(os.path.join(data_dir_DA, 'consider_names.npy'))
consider_images_pred = np.load(os.path.join(data_dir_DA, 'consider_images_pred.npy'))
consider_names.shape, consider_images_pred.shape

In [ ]:
plt.hist(consider_images_pred[:,0],bins=40, label = 'nv',alpha=0.3)
plt.title("DA-Real")
plt.legend()
plt.show()

In [ ]:
plt.hist(all_fake_pred[:,0],bins=40, label = 'Predicted probability on Fake images', alpha=0.3)
plt.hist(consider_images_pred[:,0],bins=40, label = 'Predicted probability on Real images',alpha=0.3)
plt.hist(all_fake_cond[:,0],bins=40, label = 'Condition used for creating Fake images',alpha=0.3)
plt.title("Data Augmentation consider for re-training: Real + Fake images")
plt.ylim([0,300])
plt.legend(bbox_to_anchor=(1.05, 1))
plt.show()

In [ ]:
dataset = SKIN(file_names=consider_names, transforms=transforms)
loader = data.DataLoader(
    dataset,
    batch_size=batch_size,
    drop_last=True,
    shuffle=False
)

In [ ]:
data_dir_save

In [ ]:
all_real_pred = np.empty([0])
all_real_imgs = np.empty([0])
all_real_imgs_d = np.empty([0])
for samples in tqdm(loader): 
    real_imgs = samples['img']
    real_imgs = real_imgs.to(device)
    cls_preds, cls_pred_f = classifier(real_imgs)
    cls_preds = torch.sigmoid(cls_preds)
    real_img_np = np.asarray(real_imgs.detach().cpu())
    cls_pred_np = np.asarray(cls_preds.detach().cpu())
    if all_real_imgs.shape[0] == 0:
        all_real_pred = cls_pred_np
        all_real_imgs = real_img_np
    else:
        all_real_pred = np.append(all_real_pred, cls_pred_np,axis=0)
        all_real_imgs = np.append(all_real_imgs, real_img_np,axis=0)
    
np.save(os.path.join(data_dir_save, 'all_real_pred.npy'), all_real_pred)
np.save(os.path.join(data_dir_save, 'all_real_imgs.npy'), all_real_imgs)


In [ ]:
all_real_pred = np.load(os.path.join(data_dir_save, 'all_real_pred.npy'))
all_real_pred.shape

In [ ]:
#save fake-pool as separate files
data_dir_save_real = os.path.join(save_dir, 'save', 'real')
try:
    os.makedirs(data_dir_save_real)
except:
    pass
all_rows = []
for i in range(all_real_imgs.shape[0]):
    file_name = os.path.join(data_dir_save_real, 'real_' + str(i) + '.npy')
    np.save(file_name, all_real_imgs[i])
    row = {}
    row['names'] = file_name
    for j in range(len(pathologies)):
        row[pathologies[j]] = all_real_pred[i][j]
    all_rows.append(row)
df_real = pd.DataFrame(data=all_rows)
print(df_real.shape)
df_real.head(2)

In [ ]:
df_real.to_csv(os.path.join(data_dir_save, 'real.csv'),index=None)

# Re-Train Clf using DA

In [ ]:
seed = config['seed']
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
classifier = xrv.models.DenseNet(num_classes=config['num_classes'], in_channels=config['channel'], drop_rate = config['drop_rate'], \
                                 weights = cls_final_path, return_logit=True,\
                                 **xrv.models.get_densenet_params(config['model'])).to("cuda") 

In [ ]:
optim = torch.optim.Adam(classifier.parameters(), lr=0.00001, weight_decay=1e-5, amsgrad=True)
criterion = torch.nn.BCEWithLogitsLoss()
classifier.to("cuda")

In [ ]:
df_real = pd.read_csv(os.path.join(data_dir_save, 'real.csv'))
df_fake = pd.read_csv(os.path.join(data_dir_save, 'fake.csv'))
print(df_real.shape, df_fake.shape)

In [ ]:
# consider all real images, and sample fake images based on mix ratio
mix_ratio = 0.5
fold = 1
fold_rs = [[0,1234], [34, 456], [678, 1], [45,234], [567,12]]
num_fake_imgs = int(df_real.shape[0]*mix_ratio)
df_fake = df_fake.sample(frac=1, random_state=fold_rs[fold][0])
df_fake = df_fake.sample(frac=1, random_state=fold_rs[fold][1])
df_fake = df_fake.sample(n = num_fake_imgs)
df_final = pd.concat([df_real,df_fake])
print(df_final.shape)
print("Real: ", df_real.shape, "Fake: ", df_fake.shape, "Total: ", df_final.shape)

In [ ]:
fig = plt.figure(figsize=(5,3))
plt.hist(df_real[config['class_names'][0]],bins=40, label = 'real')
plt.hist(df_fake[config['class_names'][0]], bins=40, label = 'fake')
plt.title('DA - Mix Ratio: 0.2')
plt.legend()
plt.show()

In [ ]:
from torch.utils.data import Dataset

class SKIN(Dataset):
    def __init__(self, csv, pathologies, seed=0):
        super(SKIN, self).__init__()
        np.random.seed(seed)  # Reset the seed so all runs are the same.                             

        self.csv = csv
        self.pathologies = pathologies
        self.labels = []
        for pathology in self.pathologies:
            if pathology in self.csv.columns:
                mask = self.csv[pathology]
                
                self.labels.append(mask.values)
        self.labels = np.asarray(self.labels).T
        self.labels = self.labels.astype(np.float32) 
    
    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):          
        img_path = self.csv.iloc[idx]["names"]
        img =torch.from_numpy(np.load(img_path))
        return {"img":img, "lab":self.labels[idx], "idx":idx, "file_name":img_path}

In [ ]:
dataset = SKIN(csv=df_final, pathologies=config['class_names'])
loader = data.DataLoader(
    dataset,
    batch_size=32,
    drop_last=True,
    shuffle=True
)

In [ ]:
for e in range(10):
    classifier.train()
    #requires_grad(classifier, True)
    for batch in tqdm(loader):
        targets = batch['lab'].to(device)
        real_img = batch['img'].to(device)
        outputs,_ = classifier(real_img)
        loss = torch.zeros(1).to(device).float()
        for task in range(targets.shape[1]):
            task_output = outputs[:,task]
            task_target = targets[:,task]
            task_loss = criterion(task_output.float(), task_target.float())
            loss += task_loss
        loss = loss.sum()
        loss.backward()
        optim.step()
    torch.save(classifier, os.path.join(data_dir_DA, 'e_'+str(e+1)+ '_mix_ratio_' + str(mix_ratio)+\
                                        '_fold' +str(fold)+'.pt'))
    classifier.eval()
    

In [ ]:
# torch.save({'state_dict' : classifier.state_dict(), 'use_se':True},
#             os.path.join(data_dir_DA, 'e_'+str(e+1)+ '_mix_ratio_' + str(mix_ratio)+\
#                                         '_fold' +str(fold)+'.pt'))

In [ ]:
# torch.load(os.path.join(data_dir_DA, 'e_'+str(e+1)+ '_mix_ratio_' + str(mix_ratio)+\
#                                         '_fold' +str(fold)+'.pt'))

In [ ]:
!nvidia-smi

In [ ]:
import pickle
import seaborn as sns
import matplotlib.pyplot as plt  
from tqdm import tqdm   
import pandas as pd
import numpy as np
import os, sys, pdb
import torch, torchvision
import pickle
import time
from PIL import Image

sys.path.insert(0,'/jet/home/nmurali/asc170022p/nmurali/projects/augmentation_by_explanation_eccv22/Classifier')
import models, datasets
sys.path.insert(0,'/jet/home/nmurali/asc170022p/nmurali/projects/shortcut_detection_and_mitigation/scripts')
from utils import *

dir_path = '/jet/home/nmurali/asc170022p/nmurali/projects/shortcut_detection_and_mitigation/data/nih/layer_embeddings/imgsize_128/nih_spurious_iterations'
for filename in tqdm(os.listdir(dir_path)):
    
    pkl_path = os.path.join(dir_path,filename)
    ranks = []
    
    try:
        with open(pkl_path, 'rb') as handle:
            info_dict = pickle.load(handle)
            for feat in tqdm(info_dict['feats']):
                ranks.append(int(torch.linalg.matrix_rank(feat)))

        with open(os.path.join(dir_path,'layer_ranks',filename), 'wb') as handle:
            pickle.dump(ranks, handle)
            del feat, info_dict
            torch.cuda.empty_cache()  
    except:
        pass

 21%|██        | 6/29 [00:10<00:38,  1.68s/it]


 45%|████▍     | 13/29 [00:21<00:21,  1.34s/it]


 69%|██████▉   | 20/29 [00:24<00:09,  1.09s/it]


 93%|█████████▎| 27/29 [00:30<00:01,  1.12it/s]


 10%|█         | 3/29 [00:06<01:01,  2.36s/it]


...

  0%|          | 0/29 [00:00<?, ?it/s]


 24%|██▍       | 7/29 [00:12<00:36,  1.68s/it]


 48%|████▊     | 14/29 [00:22<00:19,  1.30s/it]


 72%|███████▏  | 21/29 [00:25<00:08,  1.12s/it]


 65%|██████▌   | 64/98 [2:25:01<1:22:15, 145.17s/it]